In [0]:
%pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
#source link - http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz
plotSummaries = sc.textFile('/FileStore/tables/plot_summaries.txt')
#creating a dataframe, converting it to rdd, and mapping over RDD
moviesMetadata = spark.read.csv("/FileStore/tables/movie_metadata.tsv", sep=r'\t', header=False)
movies = moviesMetadata.rdd 
movieNames = movies.map(lambda row : (row._c0, row._c2))
movies = movieNames.collectAsMap()

In [0]:
#getting the key-value tupple pairs
summaries = plotSummaries.map(lambda line : line.split('\t')).map(lambda x: (x[0],x[1])).map(lambda x:(x[0],x[1].lower())).map(lambda x : (x[0], x[1].split(" ")))
summaries.take(5)

Out[19]: [('23890098',
  ['shlykov,',
   'a',
   'hard-working',
   'taxi',
   'driver',
   'and',
   'lyosha,',
   'a',
   'saxophonist,',
   'develop',
   'a',
   'bizarre',
   'love-hate',
   'relationship,',
   'and',
   'despite',
   'their',
   'prejudices,',
   'realize',
   'they',
   "aren't",
   'so',
   'different',
   'after',
   'all.']),
 ('31186339',
  ['the',
   'nation',
   'of',
   'panem',
   'consists',
   'of',
   'a',
   'wealthy',
   'capitol',
   'and',
   'twelve',
   'poorer',
   'districts.',
   'as',
   'punishment',
   'for',
   'a',
   'past',
   'rebellion,',
   'each',
   'district',
   'must',
   'provide',
   'a',
   'boy',
   'and',
   'girl',
   '',
   'between',
   'the',
   'ages',
   'of',
   '12',
   'and',
   '18',
   'selected',
   'by',
   'lottery',
   '',
   'for',
   'the',
   'annual',
   'hunger',
   'games.',
   'the',
   'tributes',
   'must',
   'fight',
   'to',
   'the',
   'death',
   'in',
   'an',
   'arena;',
   'the',
   'sole',

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Out[3]: True

In [0]:
#return words which are not stopwords
import string
from nltk.corpus import stopwords
def removeStopWords(words):
    stopWords = list(stopwords.words('english'))
    puncts = string.punctuation+' '
    ans = []
    for word in words:
        word = word.strip(puncts)
        if len(word) == 0:
            continue
        if word not in stopWords:
            ans.append(word)
    return ans

In [0]:
#get the final list of valid words
def getFinalWords(summary):
    #words = summary.lower().split(" ")
    finalWords = removeStopWords(summary)
    return finalWords

In [0]:
#update summaries with the list of proper words
summaries=summaries.map(lambda x : (x[0], getFinalWords(x[1])))
summaries.take(5)

#summaries.map(lambda x : (x[0], getFinalWords(x[1])))

Out[23]: [('23890098',
  ['shlykov',
   'hard-working',
   'taxi',
   'driver',
   'lyosha',
   'saxophonist',
   'develop',
   'bizarre',
   'love-hate',
   'relationship',
   'despite',
   'prejudices',
   'realize',
   'different']),
 ('31186339',
  ['nation',
   'panem',
   'consists',
   'wealthy',
   'capitol',
   'twelve',
   'poorer',
   'districts',
   'punishment',
   'past',
   'rebellion',
   'district',
   'must',
   'provide',
   'boy',
   'girl',
   'ages',
   '12',
   '18',
   'selected',
   'lottery',
   'annual',
   'hunger',
   'games',
   'tributes',
   'must',
   'fight',
   'death',
   'arena',
   'sole',
   'survivor',
   'rewarded',
   'fame',
   'wealth',
   'first',
   'reaping',
   '12-year-old',
   'primrose',
   'everdeen',
   'chosen',
   'district',
   '12',
   'older',
   'sister',
   'katniss',
   'volunteers',
   'take',
   'place',
   'peeta',
   'mellark',
   "baker's",
   'son',
   'gave',
   'katniss',
   'bread',
   'starving',
   'district',
   '

In [0]:
#total number of movies
N = summaries.count()
N

Out[24]: 42306

In [0]:
# calculating tij which is count of ith word in a jth movie document
summary = summaries.flatMapValues(lambda x : x)
tij = summary.map(lambda t : ((t[0], t[1]), 1)).reduceByKey(lambda x, y : x+y)
tij.take(5)

Out[27]: [(('9252321', 'contradictory'), 1),
 (('1925869', 'near'), 1),
 (('5414895', 'night'), 1),
 (('18369853', 'rope'), 1),
 (('529276', 'look'), 1)]

In [0]:
# calculating count of a ith word in all the movie documents
ni = summary.distinct().map(lambda t : (t[1], t[0])).reduceByKey(lambda x, y : x+" "+y).map(lambda t : (t[0], len(t[1].split(" "))))
ni = ni.collectAsMap()

In [0]:
import math
tfIdf = tij
tfIdf = tfIdf.map(lambda x : (x[0], float(x[1]*float(math.log(N/ni[x[0][1]]))))).sortBy(lambda x : -x[1])
tfIdf = tfIdf.collectAsMap()

In [0]:
tfIdf

Out[37]: {('21768047', 'keroro'): 991.503712647511,
 ('6283620', 'hyun'): 949.5408600838645,
 ('6283620', 'soo'): 939.2580213502184,
 ('22175264', 'timmy'): 837.9563369335178,
 ('15465392', 'lucian'): 822.9354120726825,
 ('7548602', 'marcellus'): 818.4367606898276,
 ('4457831', 'morgaine'): 809.6039991199793,
 ('6283620', 'eun'): 742.9310033533852,
 ('28443267', 'luk'): 711.5791405531826,
 ('4969625', 'orry'): 703.0771571305083,
 ('3926180', 'franklin'): 692.1518727474786,
 ('11467890', 'sabata'): 687.893177540087,
 ('11584626', 'hyeon-su'): 671.119104533667,
 ('15769706', 'vamsi'): 669.2002252019616,
 ('2650227', 'janie'): 665.4244736118126,
 ('10619079', 'genocyber'): 660.4664203347199,
 ('21673015', 'ged'): 651.0703908640374,
 ('25092646', 'obie'): 637.4103691767776,
 ('35581515', 'hushpuppy'): 617.8556835389315,
 ('2065944', 'shogo'): 617.4912951400033,
 ('3257858', 'gidget'): 614.9407474828836,
 ('23325348', 'cherie'): 601.0220151998877,
 ('20887118', 'cavil'): 596.5503151410373,


In [0]:
movies

Out[38]: {'975900': 'Ghosts of Mars',
 '3196793': 'Getting Away with Murder: The JonBenét Ramsey Mystery',
 '28463795': 'Brun bitter',
 '9363483': 'White Of The Eye',
 '261236': 'A Woman in Flames',
 '13696889': 'The Gangsters',
 '18998739': "The Sorcerer's Apprentice",
 '10408933': "Alexander's Ragtime Band",
 '9997961': 'Contigo y aquí',
 '2345652': 'City of the Dead',
 '175026': 'Sarah and Son',
 '24229100': 'Lady Snowblood 2: Love Song of Vengeance',
 '6631279': 'Little city',
 '171005': 'Henry V',
 '18296435': 'Aaah Belinda',
 '11250635': 'The Mechanical Monsters',
 '30388930': '1919',
 '77856': 'Mary Poppins',
 '32456683': 'Die Fahne von Kriwoj Rog',
 '33420460': 'Keep the Change',
 '175024': "The Devil's Holiday",
 '612710': 'New Rose Hotel',
 '21926710': 'White on Rice',
 '33427105': 'Freddy and the Song of the South Pacific',
 '31983669': 'Road to Life',
 '17715326': 'Camera Thrills',
 '22087420': 'Ferdinando I, re di Napoli',
 '20604092': 'Anbu Thozhi',
 '21344842': 'Middle A

In [0]:
def singleSearch(query, topHitsCount=10):
    output = []
    for values in tfIdf.keys():
        if values[1] == query:
            output.append(values[0])
    ans = []
    for val in range(min(len(output), topHitsCount)):
        ans.append(movies[output[val]])
    print("Results : "+str(ans))

In [0]:
from scipy import spatial
def multipleSearch(query, topHitsCount=10):
    qcount = {}
    for q in query:
        if q not in qcount.keys():
            qcount[q] = 1
        else:
            qcount[q]+=1
    words = []
    qVector = []
    for key, value in qcount.items():
        words.append(key)
        qVector.append(value)

    movieId = list(movies.keys())
    cosine = {}
    for doc in movieId:
        dVector = []
        for word in words:
            t = (doc, word)
            if t in tfIdf:
                dVector.append(tfIdf[t])
            else:
                dVector.append(0)
        s = set(dVector)
        if len(s) == 1 and 0 in s:
            continue
        result = 1 - spatial.distance.cosine(qVector, dVector)
        cosine[doc] = result
    
    sortedHits = list(sorted(cosine.items(), key=lambda item: -item[1])) # list maintains the order of sort
    sortedHits = sortedHits[0:min(len(sortedHits), topHitsCount)]
    ans=[]
    for k in range(len(sortedHits)):# getting movie names for the resultant movie ids
        temp_key = sortedHits[k][0] # movie id
        #sortedHits[k]= (movies[temp_key], sortedHits[k][1])
        ans.append(movies[temp_key])
    print("Results : "+str(ans))

In [0]:
query_file = spark.read.csv("/FileStore/tables/queries_1-1.csv") # all search queries stored at first row of the csv file with delimiter','
queries=[]
for i in query_file.collect()[0]:
    queries.append([getFinalWords(i.strip().lower()),i])
for query in queries:
    print("\n")
    print('For Query "'+ query[1]+'"')
    if len(query[0])==0:
        print("Invalid Query")
        continue
    
    if len(query[0])>1:
        multipleSearch(query[0])
    else:
        singleSearch(query[0][0])



For Query "Horror"
Results : ['The Hoax', 'Amma', 'An American Tail: Fievel Goes West', 'Yellow Hair 2', 'Suyamvaram', 'Runaway', 'The American Astronaut', "Bugs' Bonnets", 'Jocks', 'Natural City']


For Query " comedy"
Results : ['Bury My Heart at Wounded Knee', 'Halls of Montezuma', 'Matewan', 'Mother Goose Goes Hollywood', 'Our Mr. Sun', 'Film', 'No Time for Sergeants', 'Eternal Family', 'Scary Movie 2', 'The Big Town']


For Query " love story"
Results : ['Pava Mannippu', 'The Animatrix', 'Provoked', 'Kungumam', '2001: A Space Odyssey', "All the President's Men", 'Chakravarthi Thirumagal', 'Bat*21', 'Ganga Ki Kasam', 'Kaalia']


For Query " sci-fi"
Results : ['Breaker Morant', 'Time Out for Lessons', 'Professor', 'No Time for Sergeants', 'Eternal Family', 'Problem Child 3: Junior in Love', 'The Three Musketeers', 'Scary Movie 2', 'The Big Town', 'The Hidden']


For Query " lame"
Results : ['Bury My Heart at Wounded Knee', 'Dreams on Spec', 'Apocalypse Now', 'Hip Hip-Hurry!', 'E',